Segmenting and Clustering Neighborhoods in Toronto

Import library

In [1]:
import urllib.request
!conda install -c anaconda beautifulsoup4 bs4 --yes
from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.24  |                0         133 KB  anaconda
    bs4-4.9.1                  |                0           4 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    beautifulsoup4-4.9.1       |           py36_0         168 KB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following NEW packages will be INSTALLED:

    bs4:             4.9.1-0           anaconda

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1               --> 4.9.1


The following command must be run outside of the IPython shell:

    $ pip install html5lib

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


Choosing wiki page and get table.

In [11]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "html.parser")

Torontotable=soup.find('table', class_='wikitable sortable')

Get the data and transform the data into a pandas dataframe.

In [18]:
A=[]
B=[]
C=[]

for row in Torontotable.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighbrhood']=C

Ignore cells with a borough that is Not assigned.

In [23]:
df = df[~df.Borough.str.contains('Not assigned')]

,PostalCode,Borough,Neighbrhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Combine neighborhoods of the same postal code areas into one row with the neighborhoods separated with a comma.

In [25]:
df = df.groupby('PostalCode').agg({'Borough':'first','Neighbrhood':', '.join}).reset_index()

,PostalCode,Borough,Neighbrhood
0,M1B,Scarborough,"Malvern, Rouge\n"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park\n"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge\n"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


Assigne borough to the neighborhoods which are "Not assigned".

In [15]:
import numpy as np
df['Neighbrhood'] = np.where(df['Neighbrhood'] == 'Not assigned', df['Borough'], df['Neighbrhood'])

Print the number of rows of the dataframe.

In [88]:
df.shape

(103, 3)